# Analysis of the imaging experiment
mouse name: 059  
task: NearFar  
experimentalist: Snezana Raus-Balind  

### 1. Load the necessary object class
We use a custom-made class, ImagingSessionData, that will contain all behavioral and imaging data. We also load the matplotlib widgets to make graphis interactive under the notebook environment.

In [1]:
from ImageAnal import *
%matplotlib widget

### 2. Tell python where the data is
The required file structure is the following:  
- All data should be in the data folder
- Within the data folder separate subfolders are needed for each mouse. Folder name starts with the **name** of the mouse.
- For each mouse there should be at least two folders: one for the **imaging** data and one for the **behavioral** data.
- The behavioral folder is named as `MouseName_TaskName` - so we need a separate folder for each different task
- The behavioral log files are in separate subfolders named by the experiment's start time within the behavioral folder - e.g. `2021-02-03_10-15-50`
- The imaging folder is named as `MouseName_imaging`
- The suite2p imaging files are also in separate folders for each experiment below the imaging folder. 

In [2]:
datapath = os.getcwd() + '/' #current working directory - look for data and strings here!
date_time = '2021-02-03_10-15-50' # date and time of the imaging session
name = 'srb059' # mouse name
task = 'NearFar' # task name

## locate the suite2p folder
suite2p_folder = datapath + 'data/' + name + '_imaging/Suite2P_4_19-05-2021/'

## the name and location of the imaging log file
imaging_logfile_name = suite2p_folder + 'srb059_TSeries-02032021-1011-001.xml'

## the name and location of the trigger voltage file
TRIGGER_VOLTAGE_FILENAME = suite2p_folder + 'srb059_TSeries-02032021-1011-001_Cycle00001_VoltageRecording_001.csv'

### 3. Load all the data - this taks ~20 secs in my computer
Python looks for the data in the specified folders. It loads the behavioral data (position, lick and rewards) as well as the imaging data. It calculates the activity of the cells as a function of position in the different corridors and calculates their spatial tuning measures and corridor selectivity.  
The name of the object that contains all the data is `D1` here - Data 1.

In [3]:
# 3. load all the data - this taks ~20 secs in my computer
D1 = ImagingSessionData(datapath, date_time, name, task, suite2p_folder, imaging_logfile_name, TRIGGER_VOLTAGE_FILENAME)#, startendlap=[27, 99])


trigger logfile loaded
trigger voltage signal loaded
triggers after: 22
n_extra_indexes 5
candidate log indexes [0, 99, 178]
min recorded trigger length: 0.012000000000000455
relevant behavior located, lap time of the first frame: 870.993125 , log reference index: 99
slight warning - testing some late candidates failed
suite2p data loaded
corrected offset: 870.9851249999999 voltage_delay: 0.0080000000000382
suite2p time axis loaded
calculating dF/F and SNR...


/Users/ubi/Projects/KOKI/VR/MiceData/ImageAnal.py:408: RuntimeWarning: invalid value encountered in true_divide
  self.dF_F[i_cell,] = (self.F[i_cell,] - baseline) / baseline


SNR done
dF/F calculated for cell ROI-s
ExpStateMachineLog time interval > 1s:  10  times
[   5.03966018  178.07813661  184.49403693  186.90899334  207.4688484
  213.4257944   217.7201534   392.68928535  653.35621006 1802.07914619]
calculating rate, reliability and Fano factor...
calculating Skaggs spatial info...
calculating proportion of active laps...
calculating proportion of active laps based on dF/F ...
calculating linear tuning specificity ...
calculating rate, reliability and Fano factor...


/Users/ubi/Projects/KOKI/VR/MiceData/utils.py:17: RuntimeWarning: invalid value encountered in true_divide
  r = np.divide(r_num, r_den, out=out_vec, where=vec_nonzero)


calculating Skaggs spatial info...
calculating proportion of active laps...
calculating proportion of active laps based on dF/F ...
calculating linear tuning specificity ...
calculating corridor selectivity ...
calculating corridor similarity ...


The behavior is divided into laps (trials or runs). You can check the **number of laps** and which lap is associated with imaging data in the following way:

In [4]:
print(D1.n_laps)
print(D1.i_Laps_ImData)

178
[ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86
  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163 164 165 166 167]


So we have 178 laps and laps 33-167 contain imaging data.  
### 4. Plotting the behavioral data
You can plot the behavioral data of the session:

In [5]:
D1.plot_session(selected_laps=D1.i_Laps_ImData)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 5. Plot the ratemaps
First, we plot the ratemaps of some neurons. There are several options - selecting the cells, sorting and normalising the ratemaps.
- selection: here we select all active cells (activity in at least 20% of laps), but any other selection criteria 
- sorting: place fields can be sorted by either corridors
- place cells can be normalised - so the peak has the same height

In [6]:
cellids = np.nonzero(((D1.cell_activelaps[0]>0.2) + (D1.cell_activelaps[1]>0.2)))[0]
D1.plot_ratemaps(cellids = cellids, sorted=True, corridor_sort=3, normalized=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 6. Plot the spatial properies of the neurons

In [7]:
D1.plot_properties(cellids=cellids, interactive=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 7. Calculate significance of tuning by shuffling the imaging data 

In [8]:
D1.calc_shuffle(cellids, 1000, 'shift', batchsize=50)

loading shuffling P-values from file...
0
1
selective cells: [  3   5  15  16  23  26  29  33  34  35  36  56  68  72  74  76  77  79
  82  84  85  87  88  89  93  95 107 115 122 125 126 141 143 144 152 154
 158 160 162 166 168 172 182 187 191 195 211 213 217 231 233 234 235 239
 250 251 253 265 266 269 273 280 281 282 287 290 292 295 299 300 301 302
 307 311 314 315 320 321 326 328 330 349 359 363 370 371 376 379 380 385
 388 396 397 399 401 406 419 422 424 427 429 430 433 436 442 446 466 471
 475 476 488 501 505 518 521 526 533 545 548 555 556 559 562 563 564 568
 569 581 585 589 590 595 597 603 613 654 659 661 683 686 687 693 695 699
 702 706 708 712 714 718 736 739 753 762 780 830 844 846 855 861 878 888
 891 892 903 910 913 918 992]
similar cells: [ 13  89 106 185 221 310 313 320 327 357 409 468 521 571 818 913]


In [13]:
print(D1.tuned_cells)

[array([   2,    5,    8,    9,   10,   12,   13,   14,   16,   19,   22,
         23,   24,   26,   27,   28,   30,   31,   35,   36,   38,   40,
         41,   42,   43,   45,   46,   47,   49,   51,   52,   56,   65,
         66,   68,   69,   70,   71,   72,   73,   76,   77,   78,   79,
         80,   81,   85,   86,   87,   88,   89,   90,   92,   94,   95,
         98,   99,  100,  103,  104,  107,  109,  110,  112,  115,  117,
        119,  120,  121,  122,  125,  128,  132,  133,  134,  136,  137,
        141,  142,  143,  144,  145,  147,  148,  149,  151,  152,  153,
        154,  156,  157,  158,  159,  160,  161,  162,  163,  168,  170,
        171,  172,  174,  175,  177,  178,  179,  185,  187,  189,  190,
        191,  193,  194,  195,  198,  200,  201,  204,  208,  211,  212,
        213,  215,  217,  220,  221,  223,  224,  226,  229,  231,  233,
        234,  235,  236,  237,  239,  240,  241,  242,  243,  249,  252,
        256,  259,  264,  265,  267,  269,  274,  

In [14]:
print(len(cellids))

680


In [15]:
print(D1.similar_cells)

[ 13  89 106 185 221 310 313 320 327 357 409 468 521 571 818 913]


### 8.  Plot the activity lap by lap
We can also plot the lap by lap activity of a selected cell. Again, there are several options, but the simplest is to plot the rate as a function of position.

In [19]:
D1.plot_cell_laps(cellid=89, signal='rate') ## look at lap 20

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
D1.save_data()

Session parameters written into file:  cell_properties_corridor_3_N1022.csv
cell properties for corridor 3  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/srb059_imaging/Suite2P_4_19-05-2021/analysed_data/cell_properties_corridor_3_N1022.csv
Session parameters written into file:  cell_properties_corridor_4_N1022.csv
cell properties for corridor 4  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/srb059_imaging/Suite2P_4_19-05-2021/analysed_data/cell_properties_corridor_4_N1022.csv
Session parameters written into file:  ratemaps_corridor_3_N1022.csv
ratemap for corridor 3  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/srb059_imaging/Suite2P_4_19-05-2021/analysed_data/ratemaps_corridor_3_N1022.csv
Session parameters written into file:  ratemaps_corridor_4_N1022.csv
ratemap for corridor 4  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/srb059_imaging/Suite2P_4_19-05-2021/analysed_data/ratemaps_corridor_4_N1022.csv
Session parameters writt

Session parameters written into file:  lapdata_lap_146_N1022.csv
Session parameters written into file:  lapdata_lap_147_N1022.csv
Session parameters written into file:  lapdata_lap_148_N1022.csv
Session parameters written into file:  lapdata_lap_149_N1022.csv
Session parameters written into file:  lapdata_lap_150_N1022.csv
Session parameters written into file:  lapdata_lap_151_N1022.csv
Session parameters written into file:  lapdata_lap_152_N1022.csv
Session parameters written into file:  lapdata_lap_153_N1022.csv
Session parameters written into file:  lapdata_lap_154_N1022.csv
Session parameters written into file:  lapdata_lap_155_N1022.csv
Session parameters written into file:  lapdata_lap_156_N1022.csv
Session parameters written into file:  lapdata_lap_157_N1022.csv
Session parameters written into file:  lapdata_lap_158_N1022.csv
Session parameters written into file:  lapdata_lap_159_N1022.csv
Session parameters written into file:  lapdata_lap_160_N1022.csv
Session parameters writte